## Use lib

In [23]:
import numpy as np
import  pandas as pd
import os
from konlpy.tag import Okt 
okt = Okt()
from tqdm.notebook import tqdm
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import copy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


from tqdm import tqdm
import time

############## 머신러닝 모델#########################
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()



############## 평가 ###############################
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV

In [ ]:
os.chdir('G:\내 드라이브\제주_전기차\data')

## Use Data

In [ ]:
# df_test2 = pd.read_csv('블로그전기차문장.csv')
# df_train = pd.read_csv('전기차_전처리및토큰화.csv')

## 전처리

In [62]:
la = LabelEncoder()

In [63]:
la.fit(y_train)

LabelEncoder()

In [75]:
y_train_la = la.fit_transform(y_train)
y_test_la = la.transform(y_test)

In [46]:
df_test.전기차문장 = df_test.전기차문장.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
df_test.전기차문장.shape

C:\Users\alsdn\AppData\Local\Temp/ipykernel_2580/519594794.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test.전기차문장 = df_test.전기차문장.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')


(8719,)

In [47]:
df_test.isnull().sum()

제목           0
날짜           0
내용           0
긍부정          0
href         0
year         0
lr긍부정        0
lr긍부정이진분류    0
감성사전긍부정      0
전기차문장        0
dtype: int64

In [48]:
df_train.isnull().sum()

전처리    0
평가     0
dtype: int64

In [73]:
stop_text = '은 는 이 가 을 를 의 에게 에 들 좀 잘 과 도 으로 자 와 ㅋㅋ ㅠㅠ ㅎㅎ'
stopwords = stop_text.split()
X_val = []
for sentence in tqdm(df_test.전기차문장):
    morphs = okt.morphs(sentence, stem=True)
    tmp_list = [word for word in morphs if word not in stopwords]
    tmp_str = ' '.join(tmp_list)
    X_val.append(tmp_str)

100%|██████████| 8719/8719 [04:08<00:00, 35.05it/s] 


In [51]:
X_train,X_test,y_train,y_test = train_test_split(
    df_train.전처리,df_train.평가,stratify=df_train.평가,random_state=2022
)

In [52]:
X_train.shape,X_test.shape

((296083,), (98695,))

In [77]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer()
cvect.fit(X_train)

CountVectorizer()

In [78]:
X_train_cv=cvect.transform(X_train)
X_test_cv=cvect.transform(X_test)
X_val_cv=cvect.transform(X_val)

In [79]:
lrc = LogisticRegression(max_iter=300)
lrc.fit(X_train_cv,y_train_la)


C:\Users\alsdn\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=300)

In [72]:
type(X_test)

pandas.core.series.Series

In [80]:
lrc.score(X_test_cv,y_test_la)

0.8163230153503217

In [82]:
la.inverse_transform(lrc.predict(X_val_cv))

array(['P', 'N', 'P', ..., 'E', 'P', 'P'], dtype=object)

In [85]:
df_test['전기차문장']=la.inverse_transform(lrc.predict(X_val_cv))

In [87]:
df_train

,전처리,평가
0,아이오 닉 휠 베이스 가치 서비스 센터 가다 부품 수급 문제 로 렌터카 받다 오다 ...,P
1,현대차 아이오 닉 테슬라 모델 뭘 살다 요 현대차 아이오 닉 테슬라 모델 뭘 살다 ...,P
2,완판 롯데 렌터카 신 차장 테슬라 프로모션 대다 공지 드리다 스탠다드 롱 레인지 안...,P
3,코 나 충전 중 화재 로 전 손 되다 안녕하다 코 나 회원 이다 오늘 충전 중 자연...,P
4,전기차 충전 요금 원 복및 조속 한 충전 인프라 확충 대한 국민 청원 진행 하다 읽...,P
...,...,...
394773,대기 순번 문의 관련 되다 주제 대해 미리 카페 내 검색 하다 보다 네 아니다 네 ...,E
394774,오다 진짜 미쳣나봐 요 쿠팡 드디어 미쳣나봐 요 ㅋㅋㅋ 나이키 랑 아디다스 뉴발 신...,P
394775,니 로 회생제동 질문 안녕하다 니 로 중고 로 타고 있다 네비업뎃 하고 나서다 부터...,E
394776,가입 인사 드리다 자기 소개 안녕하다 기다리다 있다 지역 고양시 한글 별명 뜻 이름...,P


In [88]:
df_test

,제목,날짜,내용,긍부정,href,year,lr긍부정,lr긍부정이진분류,감성사전긍부정,전기차문장
0,2022년 4박 5일 신혼여행 후기 ing 제주도 (Feat. 일오반식당...,2022.03.17,제주도신혼여행후기내돈내산년 년 월 일 드뎌 결혼식이 끝나고 우도 가자마자 전기차를 ...,0,https://blog.naver.com/apple_378?Redirect=Log&...,2022,E,P,E,P
1,"제주공항숙소 시리우스 제주호텔: 트윈룸, 디럭스룸 장단점",2022.03.17,디럭스룸 후기 제주특별자치도 제주시 도령로 전화 공항의 지하 주차장은 지하층까지...,-1,https://blog.naver.com/zoonee96?Redirect=Log&l...,2022,E,P,F,N
2,서귀포 바닷가 한적한 마을에서 힐링을 해봅시다...,2022.03.17,수용 가능해 쾌적하고 편안하게 머무르기 좋습니다 에디터 전기차 충전기를 이용할 수...,8,https://blog.naver.com/yenyo?Redirect=Log&logN...,2022,E,P,P,P
3,[신혼여행②]제주1일차_제주로렌트카/순옥이네명가/신라호텔...,2022.03.17,신혼여행 일정은 제주도 도착 이날 아 오빠가 전기차 충전에 대해서 걱정이 많아서 ...,0,https://blog.naver.com/yumichoe9810?Redirect=L...,2022,E,P,E,N
4,"g80, s90, 벤츠e클래스, 테슬라모델y 모두 타본 시승 후기~~~!!!!",2022.03.17,제주에서 글을 쓰는 봉우리 입니다 차알아보고 계시는군요 저도 작년에 처음 시승 해본...,0,https://blog.naver.com/borasos?Redirect=Log&lo...,2022,E,P,E,E
...,...,...,...,...,...,...,...,...,...,...
8966,제주도 여행 렌트카 대여 :: 스타렌터카,2019.06.08.,렌터카 하우스 적혀있는대로 가면 됩니다 날씨가 참 좋던 제주 저 멀리 내 돈 내고...,4,https://blog.naver.com/s__1202?Redirect=Log&lo...,2019,E,P,P,P
8967,타기_쏘카 & 쏘카스테이션 제주 이용후기(Feat. 쏘카패스),2019.06.08.,같은 전기차 지프 레니게이드 등 다양한 차량이 여행객을 기다리고 있었습니다 제주전기...,1,https://blog.naver.com/aramf16?Redirect=Log&lo...,2019,E,P,P,P
8968,"♨ 제주도 6박 7일 여행 비용, 경비 다 알려드립니당 ♨",2019.06.08.,그래서 제주도를 박 일 여행을 다녀왔습니당ㅎㅎㅎㅎ 마니주 펜션 옥상에서 렌터카 차종...,0,https://blog.naver.com/star1233star?Redirect=L...,2019,E,P,E,E
8969,가볼만한 곳/제주도 숨은 명소/제주 청초밭 방문 후기,2019.06.07.,규모가 어마어마합니다 걸어서 다 못갈것 같더라고요 영주산을 바라보며 걷기 걷기도 했...,0,https://blog.naver.com/nahmeerang0?Redirect=Lo...,2019,E,P,E,P


In [ ]:
d

In [27]:
X_test_블로그본문내용

['오랜 만 바다 보다 뻥 뚫리다 좋다 같이 커플 트레이닝복 맞추다 신혼 티 팍팍 내면 서 다니다 우도 가다 전기차 타고 우도 한 바퀴 다 돌다 습 니당다 저희 호객 행위 하 사 장님 로 인하다 배 에서 내리다 어찌저찌 바로 앞 위치 한 전기차 가게 로 가다 전기차 제일 오래되다 비추다 만 더 앞 가면 캐릭터 있다 귀엽다 전기차 많다 오랜 만 바다 보다 뻥 뚫리다 좋다 같이 커플 트레이닝복 맞추다 신혼 티 팍팍 내면 서 다니다 우도 가다 전기차 타고 우도 한 바퀴 다 돌다 습 니당다 저희 호객 행위 하 사 장님 로 인하다 배 에서 내리다 어찌저찌 바로 앞 위치 한 전기차 가게 로 가다 전기차 제일 오래되다 비추다 만 더 앞 가면 캐릭터 있다 귀엽다 전기차 많다 핑크색 하늘색 베이지색 다양하다 같이 커플 트레이닝복 맞추다 신혼 티 팍팍 내면 서 다니다 우도 가다 전기차 타고 우도 한 바퀴 다 돌다 습 니당다 저희 호객 행위 하 사 장님 로 인하다 배 에서 내리다 어찌저찌 바로 앞 위치 한 전기차 가게 로 가다 전기차 제일 오래되다 비추다 만 더 앞 가면 캐릭터 있다 귀엽다 전기차 많다 핑크색 하늘색 베이지색 다양하다 우도 까지 오다 전기차 타다 추천 드리다 우도 생각 보다 커서 걷다 다니다 완전 비추다 만 더 앞 가면 캐릭터 있다 귀엽다 전기차 많다 핑크색 하늘색 베이지색 다양하다 우도 까지 오다 전기차 타다 추천 드리다 우도 생각 보다 커서 걷다 다니다 완전 비추다 전기차 타고 중간 중간 식당 카페 들리다 있다 이쁘다 바다 뷰 사진 여러 군데 에서 찍 수 있다 습 니당다 우도 오다 필수 로 먹어주다 하다 우도 땅콩 아이스크림 핑크색 하늘색 베이지색 다양하다 우도 까지 오다 전기차 타다 추천 드리다 우도 생각 보다 커서 걷다 다니다 완전 비추다 전기차 타고 중간 중간 식당 카페 들리다 있다 이쁘다 바다 뷰 사진 여러 군데 에서 찍 수 있다 습 니당다 우도 오다 필수 로 먹어주다 하다 우도 땅콩 아이스크림 땅콩 한번 볶다 그렇다 정말 고소하다 풍 미가 진하다 맛있

In [11]:
df_test.isnull().sum()

제목             0
날짜             0
내용             0
긍부정            0
href           0
year           0
lr긍부정          0
lr긍부정이진분류      0
감성사전긍부정        0
전기차문장        252
dtype: int64

In [21]:
df_train.전처리

0         아이오 닉 휠 베이스 가치 서비스 센터 가다 부품 수급 문제 로 렌터카 받다 오다 ...
1         현대차 아이오 닉 테슬라 모델 뭘 살다 요 현대차 아이오 닉 테슬라 모델 뭘 살다 ...
2         완판 롯데 렌터카 신 차장 테슬라 프로모션 대다 공지 드리다 스탠다드 롱 레인지 안...
3         코 나 충전 중 화재 로 전 손 되다 안녕하다 코 나 회원 이다 오늘 충전 중 자연...
4         전기차 충전 요금 원 복및 조속 한 충전 인프라 확충 대한 국민 청원 진행 하다 읽...
                                ...                        
394773    대기 순번 문의 관련 되다 주제 대해 미리 카페 내 검색 하다 보다 네 아니다 네 ...
394774    오다 진짜 미쳣나봐 요 쿠팡 드디어 미쳣나봐 요 ㅋㅋㅋ 나이키 랑 아디다스 뉴발 신...
394775    니 로 회생제동 질문 안녕하다 니 로 중고 로 타고 있다 네비업뎃 하고 나서다 부터...
394776    가입 인사 드리다 자기 소개 안녕하다 기다리다 있다 지역 고양시 한글 별명 뜻 이름...
394777    취소 차 문의 드리다 관련 되다 주제 대해 미리 카페 내 검색 하다 보다 네 아니다...
Name: 전처리, Length: 394778, dtype: object

In [20]:
df_test.전기차문장

0       ​ 오랜만에 바다를 보니 뻥 뚫리고 좋았어여 ㅎㅎ  같이 커플 트레이닝복으로 맞추고...
1       ​ 공항의 지하 주차장은 지하4층까지 있고 지하 1층에 전기차 충전소가 있어요 주차...
2       주소 : 제주 서귀포시 이어도로 1028-12 ​ 숙소측 설명 : 귤낭하우스는 서귀...
3       렌트카 픽업 우리는 렌트카를 할까말까 하다가 하기로 결정했고 오빠가 마침 새차 고민...
4                                                    내용없음
                              ...                        
8966    ​ 우리가 빌린 차는  전기차 니로 ​ 나는 먼저 차에 타고 구님이 차 외관 확인하...
8967    저는 쏘카패스 베타서비스 기간에 가입하여 쏘카스테이션에서도 가능하였습니다. 추가로 ...
8968    숙박 비용은 총 6박을 하여서 215,391원에 예약했습니다! 렌터카 차종 :  B...
8969    영주산을 바라보며 걷기 걷기도 했지만 일부는 전기차로 데려다 주시더라고요 제주 청초...
8970       제대로 여행후기 남겨볼게요!!! ​ 전기차 처음 타보는데  신차라그런지  너무좋음!
Name: 전기차문장, Length: 8719, dtype: object

In [97]:
df_test2.dropna(inplace=True)

In [93]:
df_test2 = la.inverse_transform(lrc.predict(X_val_cv))

In [98]:
df_test2['전기차본문감성분석']= la.inverse_transform(lrc.predict(X_val_cv))

In [106]:
df_test2.to_csv('전기차감성분석(0325).csv',index=False)